In [1]:
from llama_models.llama3.api.datatypes import (
    CompletionMessage,
    StopReason,
    SystemMessage,
    UserMessage,
)

from llama_models.llama3.reference_impl.generation import Llama

from transformers import pipeline

from typing import Optional, List, Union

import torch.distributed.run as torchrun
import os


/home/jaero/.miniforge3/envs/venv-carter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CHECKPOINT_PATH = "/home/jaero/.llama/checkpoints/Llama3.2-3B-Instruct"

os.path.exists(CHECKPOINT_PATH) and os.path.isdir(CHECKPOINT_PATH)

True

In [3]:
def get_model(
    ckpt_dir: str,
    max_seq_len: int = 512,
    max_batch_size: int = 4,
    model_parallel_size: Optional[int] = None,
) -> Llama:
    """
    Examples to run with the models finetuned for chat. Prompts correspond of chat
    turns between the user and assistant with the final one always being the user.

    An optional system prompt at the beginning to control how the model should respond
    is also supported.

    `max_gen_len` is optional because finetuned models are able to stop generations naturally.
    """
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
        model_parallel_size=model_parallel_size,
    )
    return generator


In [4]:
# Set required torch.distributed env variables
os.environ['RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"
os.environ['MASTER_ADDR'] = "127.0.0.1"
os.environ['MASTER_PORT'] = "29500"

# get llm text generator -- call once per notebook run
generator = get_model(CHECKPOINT_PATH)


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/home/jaero/.miniforge3/envs/venv-carter/lib/python3.11/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


Loaded in 4.33 seconds


In [5]:
summarizer = pipeline("summarization", model = "facebook/bart-large-cnn", device="cuda:0")
print(summarizer)


In [10]:
def summarize_conversation(
    conversation_history: List[str],
    min_length: int = 50,
    max_length: int = 300
) -> str:
    conversation_text = "\n".join(conversation_history)
    conv_word_count = len(conversation_text.split("\n"))
    if conv_word_count < max_length:
        max_length = 150
    summary = summarizer(conversation_text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']


def simple_question_answer(
    generator: Llama,
    dialog: List[Union[UserMessage, CompletionMessage, SystemMessage, StopReason]],
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_gen_len: Optional[int] = None,
) -> any:
    """_summary_

    Args:
        generator (Llama): _description_
        dialog (List[Union[UserMessage, CompletionMessage, SystemMessage, StopReason]]): _description_
        temperature (float, optional): _description_. Defaults to 0.6.
        top_p (float, optional): _description_. Defaults to 0.9.
        max_gen_len (Optional[int], optional): _description_. Defaults to None.

    Returns:
        any: _description_
    """
    result = generator.chat_completion(
        dialog,
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    for msg in dialog:
        if isinstance(msg, SystemMessage): continue
        print(f"\n{msg.role.capitalize()}: {msg.content}\n")

    out_message = result.generation
    print(f"> {out_message.role.capitalize()}: {out_message.content}")
    print("\n==================================\n")
    
    return out_message


In [14]:
# initial prompt
context = """Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.

Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]"""


system_guidance = """Follow these rules when giving responses.
Imagine you're a technical interviewer and can only give short responses.
No matter what, don't provide any answers or code to the user until they've given up. 
Don't ask the user to consider any improvements or optimizations of their approach until after they have implemented their solution in code.
If the user is unsure about your feedback, try and explain the previous response more simply.
If the user is asking for hints, first look at evaluating their approach and see if it generally solves the problem (even if it's not an optimal solution).
If the user is able to implement a solution that compiles and solves the problem, then ask how they might improve their solution based on Time and Space complexity.
Keep this back and forth of question and answering with hints going until the user has either achieved at least a semi-optimal solution or gives up and asks for the answer.
"""


In [15]:
summarized_prompt = summarize_conversation([context])
summarized_prompt

'You may assume that each input would have exactly one solution, and you may not use the same element twice. Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target. You can return the answer in any order.'

In [16]:
print(f"\nProblem: {context}")

prompt = """Hmm so I think I could iterate over each element in the list nums and do a nested for loop to find
every pair of numbers and see if they add up to the target amount. How does that sound?
"""

context_li = [f"Problem: {summarized_prompt}"]

dialog = [UserMessage(content=prompt, context=summarize_conversation(context_li)), SystemMessage(content=system_guidance)]
out_msg = simple_question_answer(generator, dialog)

context_li.append(f"User: {prompt}")
context_li.append(f"Agent: {out_msg.content}")

user_query = ""

while True:
    user_query = input()
    if "stop" in user_query.lower(): break
    dialog = [UserMessage(content=user_query, context=summarize_conversation(context_li)), SystemMessage(content=system_guidance)]
    out_msg = simple_question_answer(generator, dialog)
    
    context_li.append(f"User: {user_query}")
    context_li.append(f"Agent: {out_msg.content}")


Your max_length is set to 150, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)



Problem: Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.

Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]

User: Hmm so I think I could iterate over each element in the list nums and do a nested for loop to find
every pair of numbers and see if they add up to the target amount. How does that sound?


> Assistant: Your approach sounds good. You can use two nested loops to iterate over each element in the list and check if the difference between the target and the current element is in the list. 

However, be aware that this approach has a time complexity of O(n^2) due to t

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



User: def two_sum(li): for n in li: print(n)

> Assistant: It looks like you have a basic understanding of the problem. However, your function `two_sum` only prints the elements of the list, but doesn't actually solve the problem.

Can you think of a way to modify your function to return the indices of the two numbers that add up to the target?


